# Assignment 5 Machine Translation
### Jing Qian (jq2282)

## Step 1. Install packages and load libraries


In [0]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

In [23]:
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU

In [0]:
import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata

## Step 2. Load dataset for English to Spanish

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def load(fname):
  # Load the file using std open
  f = open(fname, 'r')
  text = []
  for line in f.readlines():
    text.append(line.replace('\n','').split('\t'))
    
  f.close()
  return text

data = load('/content/gdrive/My Drive/spa-eng/spa.txt')

In [3]:
print(data[10:15])
print(np.shape(data))

[['Fire!', '¡Disparad!'], ['Help!', '¡Ayuda!'], ['Help!', '¡Socorro! ¡Auxilio!'], ['Help!', '¡Auxilio!'], ['Jump!', '¡Salta!']]
(118964, 2)


In [21]:
# Test code for permutation. Delete in the submit version.
arr = np.arange(9).reshape(3,3)
for i in range(5):
  np.random.seed(10)
  b = np.random.permutation(arr) # permutation not in space; shuffle in space
  print(arr,b)

[[0 1 2]
 [3 4 5]
 [6 7 8]] [[0 1 2]
 [6 7 8]
 [3 4 5]]
[[0 1 2]
 [3 4 5]
 [6 7 8]] [[0 1 2]
 [6 7 8]
 [3 4 5]]
[[0 1 2]
 [3 4 5]
 [6 7 8]] [[0 1 2]
 [6 7 8]
 [3 4 5]]
[[0 1 2]
 [3 4 5]
 [6 7 8]] [[0 1 2]
 [6 7 8]
 [3 4 5]]
[[0 1 2]
 [3 4 5]
 [6 7 8]] [[0 1 2]
 [6 7 8]
 [3 4 5]]


Start without worrying about duplicate translations
* Shuffle the dataset
* Randomly choose 5000 sentences as training, 1000 as validation.

In [0]:
np.random.seed(10)
shuffled_data = np.random.permutation(data)


## Step 2. Train a model to translate from English to Spanish
<p style="color:red;">using a couple thousand sentences.</p>

## Part 2 
Train a second model to translate between the same two languages in reverse order (say, from Spanish to English).

## Part 3
Back-translate. Use your two models to translate a sentence from English to Spanish, and then back to English. Compare the original sentence, and the back-translated sentence. Repeat this using an evaluation corpus of 1,000 sentences, and report the BLEU score.